In [1]:
%matplotlib inline
import pandas as pd
import os
import re
from Bio import SeqIO
import pysam
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import SearchIO
from pybedtools import BedTool
import numpy as np
import pybedtools
import multiprocessing
import re
import time
import matplotlib.pyplot as plt
import json
from ipywidgets import widgets
from IPython.display import display
from sklearn.externals.joblib import Parallel, delayed
import seaborn as sns
import matplotlib
import subprocess

/home/benjamin/anaconda3/lib/python3.5/site-packages/Bio/SearchIO/__init__.py:211: BiopythonExperimentalWarning: Bio.SearchIO is an experimental submodule which may undergo significant changes prior to its future official release.
  BiopythonExperimentalWarning)


In [55]:
orthofinder_all_folder = '/home/benjamin/genome_assembly/Warrior/DK0911_v04/comp_orthology/orthofinder/OrthoFinder_all'
orthofinder_three_pst_folder = '/home/benjamin/genome_assembly/Warrior/DK0911_v04/comp_orthology/orthofinder/OrthoFinder_Results_pst78_dk0911_ps104e'

In [56]:
all_df = pd.read_csv(os.path.join(orthofinder_all_folder,'Orthogroups_1.csv' ), sep='\t',index_col=0)
three_pst_df = pd.read_csv(os.path.join(orthofinder_three_pst_folder, 'Orthogroups.csv'), sep='\t',index_col=0)

/home/benjamin/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [57]:
three_pst_df.columns

Index(['DK_0911', 'Pst_104E', 'Pst_78'], dtype='object')

In [11]:
all_df.columns

Index(['Agahy', 'Atrsp', 'Chiap', 'Chicu', 'Croqu', 'Eryha', 'Hetpy', 'Leucr',
       'Melap', 'Melli', 'Mellp', 'Mereb', 'Micld', 'Micvi', 'Mixos', 'Mlr',
       'Naifl', 'Naose', 'Pucco1', 'Pucco2', 'Pucgr', 'Pucgra', 'Pucsor',
       'Pucst130', 'Pucst78', 'Pucst93210', 'Pucst93TX2', 'Puctr', 'Rhoba',
       'Rhodia', 'Rhoglu', 'Rhomi', 'Rhomuc', 'Rhosp1', 'Rhosp2', 'Rhoto1',
       'Rhoto2', 'Rhoto3', 'Rhoto4', 'Rhoto5', 'Sepsp', 'Slopil', 'Spoli',
       'Spopa', 'Sporo', 'Vda_12008', 'Vda_12158', 'Vda_12161', 'Vda_12251',
       'Vda_12253', 'Vda_17', 'Ztr_IPO323', 'Ztr_ST99CH_1A5', 'Ztr_ST99CH_1E4',
       'Ztr_ST99CH_3D1', 'Ztr_ST99CH_3D7', 'Hyabl', 'PucstDK0911'],
      dtype='object')

In [60]:
unassigned_all_df = pd.read_csv(os.path.join(orthofinder_all_folder,'Orthogroups_1_UnassignedGenes.csv' ), sep='\t',index_col=0).fillna(False)
unassigned_three_rust_df = pd.read_csv(os.path.join(orthofinder_three_pst_folder,'Orthogroups_UnassignedGenes.csv' ), sep='\t',index_col=0).fillna(False)

/home/benjamin/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,15,16,17,18,19,20,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [61]:
for species in unassigned_three_rust_df.columns:
    print('%s: %i' % (species, unassigned_three_rust_df[unassigned_three_rust_df[species] != False][species].shape[0]))

DK_0911: 2534
Pst_104E: 2991
Pst_78: 4250


In [54]:
for species in unassigned_df.columns:
    print('%s: %i' % (species, unassigned_df[unassigned_df[species] != False][species].shape[0]))

Agahy: 1186
Atrsp: 5034
Chiap: 2005
Chicu: 567
Croqu: 3062
Eryha: 1617
Hetpy: 261
Leucr: 965
Melap: 2514
Melli: 1070
Mellp: 1530
Mereb: 1798
Micld: 0
Micvi: 0
Mixos: 1158
Mlr: 249
Naifl: 4509
Naose: 3027
Pucco1: 1916
Pucco2: 1668
Pucgr: 0
Pucgra: 0
Pucsor: 3119
Pucst130: 1217
Pucst78: 807
Pucst93210: 228
Pucst93TX2: 281
Puctr: 1324
Rhoba: 362
Rhodia: 609
Rhoglu: 488
Rhomi: 808
Rhomuc: 596
Rhosp1: 55
Rhosp2: 654
Rhoto1: 374
Rhoto2: 10
Rhoto3: 364
Rhoto4: 14
Rhoto5: 22
Sepsp: 2524
Slopil: 1603
Spoli: 1594
Spopa: 686
Sporo: 199
Vda_12008: 64
Vda_12158: 20
Vda_12161: 17
Vda_12251: 27
Vda_12253: 44
Vda_17: 616
Ztr_IPO323: 621
Ztr_ST99CH_1A5: 135
Ztr_ST99CH_1E4: 147
Ztr_ST99CH_3D1: 100
Ztr_ST99CH_3D7: 365
Hyabl: 274
PucstDK0911: 705


In [62]:
gene_count_three_rust_df = pd.read_csv(os.path.join(orthofinder_three_pst_folder,'Orthogroups.GeneCount.csv' ), sep='\t',index_col=0).fillna(False)

In [66]:
gene_count_three_rust_df.columns[:3]

Index(['DK_0911', 'Pst_104E', 'Pst_78'], dtype='object')

In [69]:
gene_count_three_rust_df[gene_count_three_rust_df['DK_0911'] == 0].shape[0]

750

In [70]:
for x in gene_count_three_rust_df.columns:
    print('%s is missing in %i orthogroups' % (x, gene_count_three_rust_df[gene_count_three_rust_df[x] == 0].shape[0]))

DK_0911 is missing in 750 orthogroups
Pst_104E is missing in 648 orthogroups
Pst_78 is missing in 1369 orthogroups
Total is missing in 0 orthogroups


In [46]:
unassigned_df.PucstDK0911.fillna('False')

OG0039239           False
OG0039240           False
OG0039241           False
OG0039242           False
OG0039243           False
OG0039244           False
OG0039245           False
OG0039246           False
OG0039247           False
OG0039248           False
OG0039249           False
OG0039250           False
OG0039251           False
OG0039252           False
OG0039253           False
OG0039254           False
OG0039255           False
OG0039256           False
OG0039257           False
OG0039258           False
OG0039259           False
OG0039260           False
OG0039261           False
OG0039262           False
OG0039263           False
OG0039264           False
OG0039265           False
OG0039266           False
OG0039267           False
OG0039268           False
                 ...     
OG0094418    DK0911_20246
OG0094419    DK0911_19917
OG0094420    DK0911_27910
OG0094421    DK0911_29229
OG0094422    DK0911_25422
OG0094423    DK0911_21386
OG0094424    DK0911_27939
OG0094425   